In [1]:
import numpy as np
import pygame

class Particle:
    def __init__(self, position, velocity, particle_type, max_influence=100):
        self.position = np.array(position, dtype=float)
        self.velocity = np.array(velocity, dtype=float)
        self.particle_type = particle_type
        self.max_influence = max_influence

    def move(self, boundaries):
        self.position += self.velocity
        for i in range(2):  # Check x and y boundaries
            if self.position[i] < 0 or self.position[i] > boundaries[i]:
                self.velocity[i] *= -1  # Reverse velocity on collision


class Board:
    def __init__(self, width, height):
        self.width = width
        self.height = height
        pygame.init()
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()

    def draw_particles(self, particles):
        self.screen.fill((0, 0, 0)) 
        for particle in particles:
            x, y = int(particle.position[0]), int(particle.position[1])
            color = self.get_color_by_type(particle.particle_type)
            pygame.draw.circle(self.screen, color, (x, y), 5)  
        pygame.display.flip()

    def get_color_by_type(self, particle_type):
        colors = {
            "A": (255, 0, 0),  # Red
            "B": (0, 255, 0),  # Green
            "C": (0, 0, 255),  # Blue
            "D": (255, 255, 0)  # Yellow
        }
        return colors.get(particle_type, (255, 255, 255)) 

    def update_display(self):
        self.clock.tick(60)  


class InteractionMatrix:
    def __init__(self, types):
        self.types = types
        self.matrix = self._initialize_matrix()

    def _initialize_matrix(self):
        size = len(self.types)
        return np.random.uniform(-0.5, 0.5, (size, size))  

    def get_interaction(self, type1, type2):
        idx1 = self.types.index(type1)
        idx2 = self.types.index(type2)
        return self.matrix[idx1, idx2]

    def calculate_force(self, particle1, particle2):
        interaction_strength = self.get_interaction(particle1.particle_type, particle2.particle_type)
        distance_vector = np.array(particle2.position) - np.array(particle1.position)
        distance = np.linalg.norm(distance_vector)
        if distance == 0 or distance > particle1.max_influence:
            return np.array([0, 0])
        force = (interaction_strength / (distance ** 2)) * distance_vector 
        return force



def main():
    width, height = 800, 600
    particle_types = ["A", "B", "C", "D"]
    num_particles = 50  # Reduced for clarity

    board = Board(width, height)
    interaction_matrix = InteractionMatrix(particle_types)

    # Initialize particles with random positions and velocities
    particles = [
        Particle(
            position=np.random.rand(2) * [width, height],
            velocity=(np.random.rand(2) - 0.5) * 2,  # Random initial velocity
            particle_type=np.random.choice(particle_types)
        )
        for _ in range(num_particles)
    ]

    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        for particle in particles:
            net_force = np.array([0, 0], dtype=float)
            for other_particle in particles:
                if particle != other_particle:
                    net_force += interaction_matrix.calculate_force(particle, other_particle)
            
            # Add random movement (noise)
            noise = (np.random.rand(2) - 0.5) * 0.1
            particle.velocity += net_force + noise
            particle.move((width, height))

        board.draw_particles(particles)
        board.update_display()

    pygame.quit()

if __name__ == "__main__":
    main()



pygame 2.6.1 (SDL 2.28.4, Python 3.11.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 